In [2]:
import openai 
from qdrant_client import QdrantClient 

### Embedding function

In [3]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input = text,
        model = model,
    )
    return response.data[0].embedding

### Retrieval function

In [4]:
qdrant_client = QdrantClient(url="http://localhost:6333") 

In [24]:
def retrieve_data(query, qdrant_client, k=5): 
    query_embedding = get_embedding(query)  
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",  
        query=query_embedding, 
        limit=k,  
    ) 
    retrieved_context_ids = []
    retrieved_context = [] 
    similarity_scores = [] 
    
    for result in results.points: 
        retrieved_context_ids.append(result.payload["parent_asin"]) 
        retrieved_context.append(result.payload["description"]) 
        similarity_scores.append(result.score) 

    return {
        "retrieved_context_ids": retrieved_context_ids, 
        "retrieved_context": retrieved_context, 
        "similarity_scores": similarity_scores, 
    }


In [25]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client, k=10)

In [13]:
retrieved_context

{'retrieved_context_ids': ['B0CBMPG524',
  'B0B9FTVL58',
  'B0C142QS8X',
  'B0C6K1GQCF',
  'B0BNHVLF7G',
  'B0B14HTZ59',
  'B09WCFC5D9',
  'B0B67ZFRPC',
  'B0B7495RL6',
  'B0BS15TRJ3'],
 'retrieved_context': ['Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world

### Format retrieved context function

In [14]:
def process_context(context):
    formatted_context = "" 

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):  
        formatted_context += f"- {id}: {chunk}\n"  

    return formatted_context

In [17]:
preprocessed_context = process_context(retrieved_context)

In [16]:
print(process_context(retrieved_context))

- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether you are cycling, walking, runn

### Create Prompt function

In [19]:
def build_prompt(preprocessed_context, question): 

    prompt = f""" 
    You are a shopping assistant that can answer qeustions about the products in stock. 

    You will be given a question and a list of context. 

    Instructions: 
    - You need to answer the question based on the provided context only. 
    - Never use word context and refer to it as the available products. 

    Context: 
    {preprocessed_context} 

    Question: 
    {question} 
    """ 
    return prompt

In [22]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [20]:
print(build_prompt(preprocessed_context, "What kind of earphones can I get?"))

 
    You are a shopping assistant that can answer qeustions about the products in stock. 

    You will be given a question and a list of context. 

    Instructions: 
    - You need to answer the question based on the provided context only. 
    - Never use word context and refer to it as the available products. 

    Context: 
    - B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth 

### Generate answer function

In [21]:
def generate_answer(prompt):  
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[{"role": "system", "content": prompt}], 
        temperature=0.5, 
    ) 
    return response.choices[0].message.content


In [23]:
print(generate_answer(prompt))

You can choose from a variety of earphones, including:

1. **Open Ear Headphones**: These Bluetooth 5.3 earbuds feature a comfortable open-ear design, 60 hours of playtime, and are IPX7 waterproof, making them ideal for running and workouts.

2. **Wireless Earbuds**: These Bluetooth 5.3 headphones come with a microphone, 37 hours of playback, LED power display, deep bass, and are also IPX7 waterproof.

3. **Kids Headphones**: Designed for children, these over-ear headphones have a volume limit of 94dB for hearing protection, are foldable for easy storage, and are compatible with various devices.

4. **Bone Conduction Headphones**: These wireless headphones use bone conduction technology for sound delivery, are sweat-resistant, and have a secure fit for workouts.

5. **TWS Game Earbuds**: These true wireless earbuds are designed for gaming with low latency, deep bass, and a long battery life of up to 14 hours with the charging case.

6. **Wired Earbuds for Kids**: These cute cat-themed 

### Combined RAG Pipeline

In [26]:
def rag_pipeline(question, top_k=5): 
    qdrant_client = QdrantClient(url="http://localhost:6333") 

    retrieved_context = retrieve_data(question, qdrant_client, top_k) 
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer


In [28]:
print(rag_pipeline("Can I get come cool stuff for kids?"))

Yes, there are some cool products for kids available. You can find the TUNEAKE Kids Headphones, which are designed for comfort and safety with volume-limiting technology, making them perfect for listening to music or watching videos. Additionally, the QearFun Cat Earbuds are a fun option, featuring a cute design and a storage case, making them great gifts for school.
